In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

Loading data from Folder
In this module we will be loading the available data into the system and then saving that data in format of numpy arrays for easily loading and training various models.

In [ ]:
# Imports
import numpy
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv

In [ ]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values


# load a list of files into a 3D array of [samples, timesteps, features]
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial_Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix)
	print("Shape of training data = X : {}, y: {}".format(trainX.shape, trainy.shape))
	# load all test
	testX, testy = load_dataset_group('test', prefix)
	print("Shape of testing data = X : {}, y: {}".format(testX.shape, testy.shape))
	# zero-offset class values (Since values stat from 1, we need to start them from zero)
	trainy = trainy - 1
	testy = testy - 1
	print("Final shape of training data = X:{}, y: {}\nFinal shape of testing data = X:{}, y: {}\n".format(trainX.shape, trainy.shape, testX.shape, testy.shape))
	return (trainX, trainy), (testX, testy)
 # Load the data by calling required functions

(trainX, trainy), (testX, testy) = load_dataset('/content/drive/MyDrive/Colab Notebooks/UCI_HAR_Dataset/')

In [ ]:
trainX=trainX.reshape(7352,768)
testX=testX.reshape(2947,768)
from sklearn.preprocessing import MinMaxScaler, StandardScaler,LabelEncoder
scaler = StandardScaler()
#X_train= scaler.fit_transform(trainX)
#X_test= scaler.fit_transform(testX)

In [ ]:
import numpy as np

X=np.concatenate((trainX,testX),axis=0)
y=np.concatenate((trainy,testy),axis=0)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_test.shape

In [ ]:
!pip install imblearn
from imblearn.over_sampling import SMOTE, ADASYN


In [ ]:
X_train=X_train.reshape(7209,128,6)
X_test=X_test.reshape(3090,128,6)


In [ ]:
from tensorflow.keras.utils import to_categorical
y_train=to_categorical(Y_train)
y_test=to_categorical(Y_test)

In [ ]:
!pip install tensorflow
import tensorflow as tf # import tensorflow

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    print("Device:", tpu.master())
    strategy = tf.distribute.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print("Number of replicas:", strategy.num_replicas_in_sync)

In [ ]:
# RTA-CNN

import tensorflow as tf
from keras import  backend as K
from keras.models import Model, load_model
from keras.layers import Input, Activation, Dense, Flatten
from keras.layers import Reshape,multiply
#The following two lines were redundant, so they have been removed
#from keras.layers.merge import add, concatenate
#from keras.layers import add, concatenate
from keras.layers import Dropout
from keras.layers import Conv1D, BatchNormalization,LSTM

In [ ]:
import tensorflow as tf
from keras import  backend as K
from keras.models import Model, load_model
from keras.layers import Input, Activation, Dense, Flatten
from keras.layers import Reshape,multiply
from keras.layers import MaxPooling1D #Import MaxPooling1D directly from keras.layers
from keras.layers import MaxPooling1D
#Import add and concatenate directly from keras.layers
from keras.layers import add, concatenate
from keras.layers import Dropout
from keras.layers import Conv1D, BatchNormalization,LSTM,GRU,TimeDistributed,MultiHeadAttention
from keras.layers import ZeroPadding1D, UpSampling1D,Cropping1D, GlobalAveragePooling1D

from keras import initializers, regularizers
from keras.regularizers import l2
# imports
import keras
from keras import regularizers
from keras.models import Sequential
from keras.layers import MaxPooling1D, Conv1D, Flatten, Dense, Concatenate, GRU, Input, InputLayer,Dropout,Bidirectional
#Import plot_model from keras.utils
from keras.utils import plot_model
from keras.optimizers import *#RMSprop, Adam
from keras.callbacks import ReduceLROnPlateau , ModelCheckpoint,EarlyStopping

# Define the get_1DCNN_model function
def get_1DCNN_model():
  # Add your model definition here
  model = Sequential()
  # Example: model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(128, 1)))
  # ... add more layers
  model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(128, 6)))
  model.add(GlobalAveragePooling1D()) # Add GlobalAveragePooling1D to aggregate the output across timesteps
  model.add(Dense(6, activation='softmax')) # Output layer with 6 units (number of classes) and softmax activation
  return model

#checkPoint = ModelCheckpoint(filepath='/content/drive/My Drive/HAR/WISDM/' + model.name +'.{epoch:02d}-{loss:.2f}.h5', verbose=1, save_best_only=True)
with strategy.scope():
  model=get_1DCNN_model()
  opt = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer = opt, loss='categorical_crossentropy', metrics = ["accuracy"])
earlystopping = EarlyStopping(monitor='val_accuracy',patience=50,verbose=1,mode='auto')
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=.95, epsilon=0.001, patience=15, verbose=1)
history=model.fit(X_train, y_train, epochs=100, verbose=2, batch_size=400, validation_data=(X_test, y_test),shuffle=True,callbacks=[lr_reduce])

In [ ]:
def print_classification_report(model, X_test, y_test, title=''):
    classes = ['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING', 'LAYING']
    testX, testy = X_test, y_test#test_data
    # Get predicted and true values
    predictedY = model.predict(testX)
    y_pred = np.argmax(predictedY, axis = 1)
    y_true = np.argmax(testy, axis=1)

    print("Classification Report for '{}': \n".format(title))
    print(classification_report(y_true, y_pred, target_names=classes, digits=5))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print_classification_report(model, X_test, y_test, title='1D CNN Model')

In [ ]:
# prompt: Now plot thiss

import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


# **LSTM Model**

In [ ]:
# LSTM based Model defined layer by layer
def get_lstm_model():
    model = Sequential()
    input_shape = (128, 6)
    model.add(LSTM(100, input_shape=input_shape))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(6, activation='softmax'))
    return model

In [ ]:
import tensorflow as tf
from keras import  backend as K
from keras.models import Model, load_model
from keras.layers import Input, Activation, Dense, Flatten
from keras.layers import Reshape,multiply
# 'add' and 'concatenate' are now imported directly from keras.layers
from keras.layers import add, concatenate
from keras.layers import Dropout
from keras.layers import Conv1D, BatchNormalization,LSTM,GRU,TimeDistributed,MultiHeadAttention
from keras.layers import ZeroPadding1D, UpSampling1D,Cropping1D, GlobalAveragePooling1D
from keras.layers import MaxPooling1D # Changed line: import directly from keras.layers
from keras import initializers, regularizers
from keras.regularizers import l2
# imports
import keras
from keras import regularizers
from keras.models import Sequential
from keras.layers import MaxPooling1D, Conv1D, Flatten, Dense, Concatenate, GRU, Input, InputLayer,Dropout,Bidirectional
# 'plot_model' has been moved to 'keras.utils'
from keras.utils import plot_model
from keras.optimizers import *#RMSprop, Adam
from keras.callbacks import ReduceLROnPlateau , ModelCheckpoint,EarlyStopping
#checkPoint = ModelCheckpoint(filepath='/content/drive/My Drive/HAR/WISDM/' + model.name +'.{epoch:02d}-{loss:.2f}.h5', verbose=1, save_best_only=True)
with strategy.scope():
  model=get_lstm_model()
  opt = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer = opt, loss='categorical_crossentropy', metrics = ["accuracy"])
earlystopping = EarlyStopping(monitor='val_accuracy',patience=50,verbose=1,mode='auto')
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=.95, epsilon=0.001, patience=15, verbose=1)
history=model.fit(X_train, y_train, epochs=50, verbose=1, batch_size=400, validation_data=(X_test, y_test),shuffle=True,callbacks=[lr_reduce])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print_classification_report(model, X_test, y_test, title='LSTM Model')

In [ ]:
# prompt: Now plot this

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


In [ ]:
# prompt: Compare this model result and plot it

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Get predictions for both models
y_pred_cnn = model.predict(X_test)
y_pred_cnn = np.argmax(y_pred_cnn, axis=1)

y_pred_lstm = get_lstm_model().predict(X_test)
y_pred_lstm = np.argmax(y_pred_lstm, axis=1)

y_true = np.argmax(y_test, axis=1)

# Print classification reports
print("Classification Report for 1D CNN Model:")
print(classification_report(y_true, y_pred_cnn))

print("\nClassification Report for LSTM Model:")
print(classification_report(y_true, y_pred_lstm))

# Plot confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 1D CNN Confusion Matrix
cm_cnn = confusion_matrix(y_true, y_pred_cnn)
sns.heatmap(cm_cnn, annot=True, fmt="d", cmap="Blues", ax=axes[0])
axes[0].set_title("1D CNN Confusion Matrix")

# LSTM Confusion Matrix
cm_lstm = confusion_matrix(y_true, y_pred_lstm)
sns.heatmap(cm_lstm, annot=True, fmt="d", cmap="Blues", ax=axes[1])
axes[1].set_title("LSTM Confusion Matrix")

plt.show()
